In [7]:
#voltando ao erro de NAXIS2

#this program will combine images to make a master frame

#if you use this code, please cite Oelkers & Stassun 2018

#import the relevant libraries for basic tools
#import pyfits
import astropy.io.fits
import numpy as np
import scipy
from scipy import stats
from os import path
import math
import time
import csv

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join, exists
#from pyfits import getheader #new libraries added
from astropy.io import fits #pedro
from astropy.wcs import WCS
from astropy.io.fits import getval

ModuleNotFoundError: No module named 'pyfits'

In [8]:
###UPDATE HERE#####
#what field are you looking at?
camera = '2'
ccd = '2'
blknum = 50 #how many images go into each holder?

#this program will select the images to make the E and W master frames
#useful directories
# cdedir = '/home/thaismadruga/Documentos/IC-codigos/' #onde está o código 
# clndir = '/home/thaismadruga/Documentos/FFIs/' #onde as imagens reduzidas ficam
# rawdir = '/home/thaismadruga/Documentos/FFIs/' #onde as imagens 'cruas' ficariam 
user = 'sonic'
rawdir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/' #directory with the raw images
cdedir = '/home/'+user+'/dia_kepler/mychanges/Python/' #directory where the code 'lives'
clndir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/teste_clean/'

###END UPDATE###

#get the image list and the number of files which need reduction
files = np.loadtxt(cdedir+'images.dat',dtype = 'string',unpack = 1,usecols = (0)) 
print (files) 
nfiles = len(files) #devolve o comprimento de files

#iterate through the files
cnt = 0 #contador para o numero de imagens usadas
kk = 0 #the jumper for file placement

for ii in range(0, nfiles):
    hdulist = fits.open(rawdir+files[ii]) #no 'openfiles'
    for j in range(1,84):
        orgimg, header = fits.getdata(rawdir+files[ii], header = True)
        print(np.shape(orgimg))
        W = WCS(header)    
        

    if (ii == 0):  # get size on first iteration only
        #print clndir+files[0]
        nx = pyfits.getval(clndir+files[0], 'NAXIS2') 
        ny = pyfits.getval(clndir+files[0], 'NAXIS1')
        aa = pyfits.getheader(clndir+files[0], 1)
        #nx = aa['NAXIS2']
        #print nx
        all_data = np.ndarray(shape=(blknum,nx,ny))
        expt = np.zeros(blknum) #

        #read in the image
        img_data = pyfits.getdata(clndir+files[ii])
        expt[cnt] = pyfits.getval(clndir+files[ii],'EXPOSURE')

        #add the image to the vector
        all_data[cnt] = img_data 
        cnt = cnt+1

        if (ii % 8 == 0) and (ii > 0):
                print ('Finished with 8 images at '+str(time.strftime("%a %d %b %Y %H:%M:%S"))+'.') 
                #troquei o numero do print de 10 por 8 

        if (ii == nfiles-1) or ((ii+1) % blknum == 0):
                
                #median combine the data
                combined_data = np.median(all_data,axis=0)

                # Write data to new file    
                new_image = pyfits.PrimaryHDU(combined_data)
                new_image.header.set('NUMCOMB', cnt)
                new_image.header.set('EXPTIME', np.median(expt))

                #print the file with the appropriate counter
                if (kk < 8):
                        new_image.writeto(cdedir+'frames/'+camera+'_'+ccd+'_master_0'+str(kk)+'.fits',clobber=True)
                else: 
                        if (kk >= 8) and (kk < 100):
                                new_image.writeto(cdedir+'frames/'+camera+'_'+ccd+'_master_'+str(kk)+'.fits',clobber=True)

                print ("The master frame hold was created using a median of "+str(cnt)+" images.")
                kk = kk+1
                cnt = 0

                #clear the data file
                all_data = np.ndarray(shape=(blknum,nx,ny))
                expt = np.zeros(blknum)

del all_data, img_data # clear up some memory

OSError: /home/sonic/dia_kepler/mychanges/Python/images.dat not found.